In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt

# 1. Identifying Missing Data

In [ ]:

# We will use the orders table from Super Store for these tasks
orders = pd.read_csv('data/datasets/orders.csv')


# A. Start with some exploratory analysis methods to inspect the data
orders.columns
orders.shape
pd.DataFrame(orders.dtypes, columns=["DataTypes"])

In [ ]:
# B. Which column has the most missing data? Sort the columns by sum of null values
orders.isnull().sum().sort_values(ascending=False)

In [ ]:
# C. Looks like postal_code is our biggest problem, along with region_id
# Use a combination of filtering, isnull, and sum to count how many rows are missing both columns
orders[orders['postal_code'].isnull()]['region_id'].isnull().sum()

In [ ]:
# D. Let's drop the region_id nulls from the dataframe before proceeding
orders.dropna(subset=['region_id'], inplace=True)

In [ ]:
# E. It's the dream scenario! The IT team confirms all missing postal_code values should be 10001.0
orders['postal_code'].fillna(10001.0, inplace=True)
orders.isnull().sum()

# 2. Cleaning Our Data

In [ ]:

# A. Write a profit_margin function that accepts a row of data, which is a dictionary
#    It should return the result of dividing the profit column by the sales column (i.e. profit/sales)
def profit_margin(row):
    return row['profit'] / row['sales']

In [ ]:
# B. Create a new column in the orders dataframe called 'profit_margin' by applying the profit margin function row-by-row
orders['profit_margin'] = orders.apply(profit_margin, axis=1)

In [ ]:
# C. Use the same process to create a new column called margin_category
# If the profit_margin is less than 0, the margin_category should be "unprofitable"
# If the profit_margin is 0, the margin_category should be "break even"
# If the profit_margin is above 0, the margin_category should "profitable"
def margin_categorization(row):
    if row['profit_margin'] > 0:
        return "profitable"
    elif row['profit_margin'] == 0:
        return "break even"
    else:
        return "unprofitable"
orders['margin_category'] = orders.apply(margin_categorization, axis=1)

In [ ]:
# D. How many of our orders were unprofitable?
orders[orders['margin_category'] == "unprofitable"].shape[0]

# 3. GroupBy Insights

In [ ]:

# Segment the following data and explore aggregate values to answer the following questions

In [ ]:
# A. Which discount results in the highest mean order quantity?
orders.groupby('discount')['quantity'].mean().sort_values(ascending=False).iloc[[0]].index[0]

In [ ]:
# B. Which product has the highest mean price discount applied?
orders.groupby('product_id')['discount'].mean().sort_values(ascending=False).iloc[[0]].index[0]

# 4. Joining DataFrames

In [ ]:

# The below example joins our first two dataframes by their shared column, Symbol
openprice = pd.DataFrame({'Symbol': ['AAPL', 'DHR', 'DAL', 'AMZN'], 'OpenPrice': [217.51, 96.54, 51.45, 1703.34]})
wkhigh = pd.DataFrame({'Symbol': ['DAL', 'AMZN', 'AAPL', 'DHR'], '52wkHigh': [60.79, 2050.49, 233.47, 110.11]})
combined = pd.merge(openprice, wkhigh, how="left", left_on='Symbol', right_on='Symbol')
combined.head()

In [ ]:
# A. Join the stockname dataframe to our combined result and print the result
stockname = pd.DataFrame({'Symbol': ['AMZN', 'DHR', 'DAL', 'AAPL'], 'Name': ['Amazon', 'Danaher', 'Delta Airlines', 'Apple']})
combined = pd.merge(combined, stockname, how="left", on="Symbol")
combined.head()

In [ ]:
# B. Use the following tables from Super Store
products = pd.read_csv('data/datasets/products.csv')
orders = pd.read_csv('data/datasets/orders.csv')
returns = pd.read_csv('data/datasets/returns.csv')
regions = pd.read_csv('data/datasets/regions.csv')

In [ ]:
# i. We want to join the products and orders dataframes. 
#    Explore both dataframes to identify the common column between them
#    Use a left join to combine the tables in a dataframe named orders_with_products
orders_with_products = pd.merge(left=products, right=orders, how="left", on="product_id")
orders_with_products.columns

In [ ]:
# ii. Left join the orders_with_products and returns dataframes
orders_with_products_and_returns = pd.merge(left=orders_with_products, right=returns, how="left", on="order_id")
orders_with_products_and_returns.columns

In [ ]:
# iii. Finally, add the region data to our combined dataframe
orders_with_products_and_returns_and_regions = pd.merge(left=orders_with_products_and_returns, right=regions, how="left", on="region_id")
orders_with_products_and_returns_and_regions.columns

In [ ]:
# iv. Let's use this combined dataframe to determine the salesperson generating the most profit
orders_with_products_and_returns_and_regions.groupby('salesperson')['profit'].sum().sort_values(ascending=False).iloc[[0]].index[0]